In [1]:
# Preprocessing Step 0 - Compute height and export las tiles for clipping
# Output las files in: /n/davies_lab/Users/pbb/SelenkayDiversity/data/in/
# Note: Also added a "buffer" option to lasHeight function
#       that clips the edges of las tiles (to remover overlap). 
# PB 10/03/2022

# Updated 10/21/22
# To deal with colinear errors, implemented HagNN instead of hag delaunaya
# ran on all files
# https://pdal.io/en/stable/stages/filters.hag_nn.html?highlight=hag_nn

import sys
sys.path.append('/n/home02/pbb/scripts/halo-metadata-server/Selenkay/bin/')
from Functions import calcLasHeight, calcLasHeight_HagNN
from pathlib import Path
import time
import pdal

### Process Inside Selenkay Tiles

In [2]:
for d in ['/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220105_SelenkaySite01Inside/Terrasolid/PointCloudOutput',
          '/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220107_SelenkaySite02Inside/Terrasolid/PointCloudOutput',
          '/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220108_SelenkaySite03Inside/Terrasolid/PointCloudOutput',
          '/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220108_SelenkaySite04Inside/Terrasolid/PointCloudOutput']:
    
    p = Path(d)
    
    outpath = '/n/davies_lab/Users/pbb/SelenkayDiversity/data/in/Inside/PointClouds'
        
    for l in p.glob('*.las'):
        
        lp = Path(l)
        
        # get site name for outputting file
        site = lp.name.split('_')[0].split('Selenkay')[-1]
        op= Path(outpath + '/' + site)

        try:
            
            start = time.time()
            
            # NOTE: site 4 has a 10 m buffer on las tiles
            if ('Site04' in str(lp)):
                
                meta = calcLasHeight_HagNN(inf=l,
                                           opath=str(op),
                                           buffer=10)
                
            # The others have no buffer
            else:
                
                meta = calcLasHeight_HagNN(inf=l,
                                           opath=str(op),
                                           buffer=0)
            
            end = time.time()
            tottime = end - start

            print(f'Processed {lp.name} in {round(float(tottime), 2)} s.\n')
            
        except Exception as e:
            
            print(f"{e.__class__} for {lp.name}: \n")
            print(f"\t{e}\n")
            

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000016.las in 0.43 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000025.las in 0.0 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000012.las in 6.63 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000015.las in 478.74 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000007.las in 957.12 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000004.las in 886.35 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000002.las in 456.34 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000008.las in 826.3 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000024.las in 80.94 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000017.las in 787.81 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000019.las in 25.58 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000010.las in 513.86 s.

Processed SelenkaySite01_PointCloud_WGS84UTM37S_tile000021.las in 312.82 s.

Processed

In [3]:
# Below are tests and failed attempts to fix collinear errors
# resorted to using Hag_NN instead 10/21/22

In [4]:
# Test it
# start = time.time()
# infile = './data/in/SelenkaySite05_PointCloud_WGS84UTM37S_000002.las'
# outpath = '/n/davies_lab/Users/pbb/SelenkayDiversity/data/in/PointClouds_Outside'
# meta = calcLasHeight(inf=infile, opath=outpath)
# end = time.time()
# print(f'Took {end - start} s.\n')
# Works great!

In [5]:
# # Reprocessing individual tiles that had collinear errors 10/13/22
# # NOTE: Ended up having to use a poisson filter on the points in order to get them to run
# # Tested in cloud compare, it removed about 25% of the points for a sample tile
# # This was the only way to get hag_delauney to run without throwing a collinear error
# # it might be indicative of something else! investigate later
# # see: https://github.com/PDAL/PDAL/issues/3844

# for d in ['/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220105_SelenkaySite01Inside/Terrasolid/PointCloudOutput',
#           '/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220107_SelenkaySite02Inside/Terrasolid/PointCloudOutput',
#           '/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220108_SelenkaySite03Inside/Terrasolid/PointCloudOutput']:
    
#     p = Path(d)
    
#     outpath = '/n/davies_lab/Users/pbb/SelenkayDiversity/data/in/Inside/PointClouds'
        
#     for l in p.glob('*.las'):
        
#         lp = Path(l)
        
#         if 'Site01' in lp.name:
#             if '018.las' in lp.name:
                
#                 try:

#                     start = time.time()

#                     meta = calcLasHeight_Filter(inf=l,
#                                          opath=outpath)

#                     end = time.time()
#                     tottime = end - start

#                     print(f'Processed {lp.name} in {round(float(tottime), 2)} s.\n')

#                 except Exception as e:

#                     print(f"{e.__class__} for {lp.name}: \n")
#                     print(f"\t{e}\n")
        
#         if 'Site02' in lp.name:
#             if (('009.las' in lp.name) | ('011.las' in lp.name) | ('012.las' in lp.name) | ('016.las' in lp.name) | ('019.las' in lp.name) | ('022.las' in lp.name) | ('023.las' in lp.name) | ('024.las' in lp.name)):
                
#                 try:

#                     start = time.time()

#                     meta = calcLasHeight_Filter(inf=l,
#                                          opath=outpath)

#                     end = time.time()
#                     tottime = end - start

#                     print(f'Processed {lp.name} in {round(float(tottime), 2)} s.\n')

#                 except Exception as e:

#                     print(f"{e.__class__} for {lp.name}: \n")
#                     print(f"\t{e}\n")
                    
#         if 'Site03' in lp.name:
#             if (('023.las' in lp.name) | ('025.las' in lp.name)):
                
#                 try:

#                     start = time.time()

#                     meta = calcLasHeight_Filter(inf=l,
#                                          opath=outpath)

#                     end = time.time()
#                     tottime = end - start

#                     print(f'Processed {lp.name} in {round(float(tottime), 2)} s.\n')

#                 except Exception as e:

#                     print(f"{e.__class__} for {lp.name}: \n")
#                     print(f"\t{e}\n")


In [6]:
# for d in ['/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220107_SelenkaySite02Inside/Terrasolid/PointCloudOutput']:
    
#     p = Path(d)
    
#     outpath = '/n/davies_lab/Users/pbb/SelenkayDiversity/data/in/Inside/PointClouds'
        
#     for l in p.glob('*.las'):
        
#         lp = Path(l)
        
#         if 'Site02' in lp.name:
#             if '009.las' in lp.name:
                
#                 try:

#                     start = time.time()
                    
#                     # Note: using new added split function to break into 4 tiles
#                     # hopefully will help with collinear error
#                     meta = calcLasHeight_Split(inf=l,
#                                                 opath=outpath)

#                     # meta = calcLasHeight_Filter(inf=l,
#                     #                             opath=outpath,
#                     #                             radius=0.2)

#                     end = time.time()
#                     tottime = end - start

#                     print(f'Processed {lp.name} in {round(float(tottime), 2)} s.\n')

#                 except Exception as e:

#                     print(f"{e.__class__} for {lp.name}: \n")
#                     print(f"\t{e}\n")
           

In [7]:
# for d in ['/n/davies_lab/Lab/data/processed/Africa/Kenya/Selenkay/20220107_SelenkaySite02Inside/Terrasolid/PointCloudOutput']:
    
#     p = Path(d)
    
#     outpath = '/n/davies_lab/Users/pbb/SelenkayDiversity/data/in/Inside/PointClouds'
        
#     for l in p.glob('*.las'):
        
#         lp = Path(l)
        
#         if 'Site02' in lp.name:
#             if '009.las' in lp.name:
                
#                 try:

#                     start = time.time()
                    
#                     # Note: using new added split function to break into 4 tiles
#                     # hopefully will help with collinear error
#                     meta = calcLasHeight_HagNN(inf=l,
#                                                 opath=outpath)

#                     # meta = calcLasHeight_Filter(inf=l,
#                     #                             opath=outpath,
#                     #                             radius=0.2)

#                     end = time.time()
#                     tottime = end - start

#                     print(f'Processed {lp.name} in {round(float(tottime), 2)} s.\n')

#                 except Exception as e:

#                     print(f"{e.__class__} for {lp.name}: \n")
#                     print(f"\t{e}\n")
           